# Brain Imaging Data Structure Converter 

Converts the Net Station data into a single session of a single subject. Iterate to create a BIDS compliant data structure, Check with the BIDS validator to see if this data structure is valid.

## Import 

In [18]:
import matplotlib
import pathlib

import mne
import mne_bids

matplotlib.use('Qt5Agg')

## Create a Directory 

Rewrite this subject directory object to connect to local PCs raw lab data location.

In [19]:
subject_dir = "C:/Users/trevo/Desktop/Lab_Files/WM_eegData/"
subject_dir = pathlib.Path(subject_dir)

Rewrite this below code each time as new data is read in.

In [20]:
raw_path = subject_dir / WMfolder / subfolder / mffname
raw = mne.io.read_raw_egi(raw_path)
raw.load_data() 

Reading EGI MFF Header from C:\Users\trevo\Desktop\Lab_Files\WM_eegData\WM1\s8\WM1_isi500_s8_20200729_114544.mff...
    Reading events ...
    Assembling measurement info ...
    Synthesizing trigger channel "STI 014" ...
    Excluding events {SESS} ...
Reading 0 ... 486747  =      0.000 ...   486.747 secs...


<RawMff | signal1.bin, 137 x 486748 (486.7 s), ~508.9 MB, data loaded>

## Find and Declare Event Names 

This will label all trial_type information with these event names, found in sub-01_ses-01_task-WM1 isi0_events.tsv within the session data.

In [ ]:
pre_events = mne.find_events(raw, 'STI 014')
events = mne.pick_events(pre_events, exclude=[1, 3])
event_id = {
    'CELL': 1,
    'bgin': 2,
    'TRSP': 3,
    'fix+': 4,
    'WMD+': 5,
    'TsD-': 6
}

events #Figure out these events!

## Specify the Power Line Frequency 

In [ ]:
raw.info['line_freq'] = 60 #The hertz of the powerlines fed into the lab 

## Set EEG Reference and EOG Fields

In [ ]:
raw = raw.copy().pick_types(meg=False, eeg=True, eog=True)
raw.set_eeg_reference(ref_channels=['E129'])
raw.set_channel_types({'E8': 'eog'})
raw.set_channel_types({'E25': 'eog'})
len(raw.ch_names)
raw

## Specify Subject Information 

You will need to edit this as each new raw data input is made, look for way to streamline/automate this? Figure out a way to put initial information to participants.tsv file

In [ ]:
subject_info = {
    'initials' : initals
    #'birthday': (1970,10,1),
    #'sex': 2,
    #'hand': 2
}

raw.info['subject_info'] = subject_info
raw.info

## Write in the Data

Will list this data in participants.json and participants.tsv, as well as use it to establish the direction of the data structure.

You will need to specify the subject number, session, and task on each iteration.

In [ ]:
out_path = pathlib.Path('out_data/01_bids_structure') #Into which directory the data should go

bids_path = mne_bids.BIDSPath(subject=subject,
                              session=session,
                              task=task,
                              root=out_path) #An object that specifies the location of BIDS files

mne_bids.write_raw_bids(raw, bids_path=bids_path, events_data=events, format="BrainVision", allow_preload=True,
                        event_id=event_id) #Writes the data